# 91Trucks Chatbot using the Json Data.

In [103]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!uv pip install jq

Using Python 3.12.3 environment at: /home/dell/Desktop/Github/91_trucks_ai_ml/001_LLM_Chatbot/chatbot-uvicorn/.venv
Audited 1 package in 2ms


# Initialize the LLM Model.

In [104]:
# Import the required library and also initialize the llm model.
from langchain.llms import OpenAI
import os
from dotenv import load_dotenv
import pandas as pd
from langchain.document_loaders.json_loader import JSONLoader
from langchain.document_loaders.csv_loader import CSVLoader
import jq
import google.generativeai as genai
import warnings
warnings.filterwarnings("ignore")
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

# with the OpenAI API Key.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#  Initialize the llm.
llm = OpenAI(temperature = 0.7)
llm.invoke("who is Elon Musk and why he is so popular give me answer in structured way.").strip()

jq_schema = '''
.[] |
 {
 page_content: (
    "The truck named " + .displayName +
    " is manufactured by " + .brand +
    ". It belongs to the " + .model + " model series. " +
    "The price ranges from " + .displayPrice + ". " +
    "It is " + (if .isElectric then "an electric vehicle" else "not electric") + ". " +
    "Average rating is " + (.avgRating | tostring) + ". " +
    "It has the following variants: " + (.variants | map(.name + " at " + .displayPrice) | join(", "))
  ),
  metadata: {
    type: "vehicle",
    image: .image,
    variants: .variants,
    gallery: .gallery,
    displayName: .displayName,
    model: .model,
    brand: .brand,
    displayPrice: .displayPrice,
    isElectric: .isElectric,
    avgRating: .avgRating
  }
}
'''


# Load the json data through the langchain framework.
loader = JSONLoader(
    file_path='tata_data.json',
    jq_schema=jq_schema,
    text_content=False,
)

documents = loader.load()
print(f"✅ Loaded {len(documents)} documents.")

ValueError: jq: error: syntax error, unexpected ')' (Unix shell quoting issues?) at <top-level>, line 9:
    "It is " + (if .isElectric then "an electric vehicle" else "not electric") + ". " +                                                                             
jq: error: Possibly unterminated 'if' statement at <top-level>, line 9:
    "It is " + (if .isElectric then "an electric vehicle" else "not electric") + ". " +                
jq: 2 compile errors

In [80]:
print(type(documents[0].metadata))

<class 'dict'>


In [81]:
documents[0].page_content

'{"page_content": "Display Name: Tata 1815 LPT, Model: 1815 LPT, Brand: Tata, Price: \\u20b925.84 - \\u20b926.84 Lakh, Electric: false, Rating: 0", "metadata": {"type": "vehicle", "image": "https://images.91trucks.com/trucks/models/63/1274/tata-815-lpt-1340125921.jpg", "variants": [{"displayPrice": "\\u20b926.84 Lakh", "price": 2684000, "name": "DCR 48 HSD", "id": 2680, "value": ["Diesel"]}, {"displayPrice": "\\u20b926.00 Lakh", "price": 2600000, "name": "DCR 48 CBC", "id": 2679, "value": ["Diesel"]}, {"displayPrice": "\\u20b925.88 Lakh", "price": 2588000, "name": "DCR 42 HSD", "id": 2678, "value": ["Diesel"]}, {"displayPrice": "\\u20b925.84 Lakh", "price": 2584000, "name": "DCR 42 CBC", "id": 2677, "value": ["Diesel"]}], "gallery": ["https://images.91trucks.com/trucks/models/63/1274/tata-815-lpt-1340125921.jpg"], "displayName": "Tata 1815 LPT", "model": "1815 LPT", "brand": "Tata", "displayPrice": "\\u20b925.84 - \\u20b926.84 Lakh", "isElectric": false, "avgRating": 0}}'

In [91]:
documents[3].page_content

'{"page_content": "Display Name: Tata Yodha Crew Cabin, Model: Yodha Crew Cabin, Brand: Tata, Price: \\u20b99.52 - \\u20b912.02 Lakh, Electric: false, Rating: 4.6", "metadata": {"type": "vehicle", "image": "https://images.91trucks.com/trucks/models/63/1268/tata-yodha-crew-cabin-1307375177.jpg", "variants": [{"displayPrice": "\\u20b912.02 Lakh", "price": 1201820, "name": "4x4 VX", "id": 5106, "value": ["2200", "100 HP", "250", "52", "3990", "1200"]}, {"displayPrice": "\\u20b910.72 Lakh", "price": 1072300, "name": "SC 4X2 VX", "id": 5105, "value": ["Diesel", "2200", "D+4", "100 HP", "250", "45", "2990", "1140"]}, {"displayPrice": "\\u20b910.71 Lakh", "price": 1071250, "name": "4X4", "id": 2669, "value": ["Diesel", "100 HP", "250", "4", "2179", "45", "2990", "1060"]}, {"displayPrice": "\\u20b910.57 Lakh", "price": 1057300, "name": "4X2 HVAC", "id": 5107, "value": ["2200", "100 HP", "250", "52", "2990", "1140"]}, {"displayPrice": "\\u20b910.31 Lakh", "price": 1031230, "name": "SC 4x2", "id

# Embedding and Vector Store of the json data.

In [83]:
!uv pip install hf_xet
!uv pip install sentence-transformers faiss-cpu

Using Python 3.12.3 environment at: /home/dell/Desktop/Github/91_trucks_ai_ml/001_LLM_Chatbot/chatbot-uvicorn/.venv
Audited 1 package in 2ms
Using Python 3.12.3 environment at: /home/dell/Desktop/Github/91_trucks_ai_ml/001_LLM_Chatbot/chatbot-uvicorn/.venv
Audited 2 packages in 12ms


In [84]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore.save_local("faiss_vectorstore")

In [85]:
print(type(documents))
print(type(documents[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


# Prompt Template for the Model.

In [86]:
# prompt_template = """Given the following context and a question, generate an detailed answer based on this context only.
#     In the answer try to provide as much text as possible from the document context without making much changes.
#     If the answer is not found in the context, do internet search and give the answer in structured way.

#     CONTEXT: {context}
#     QUESTION: {question}"""


# PROMPT = PromptTemplate(
#         template=prompt_template, input_variables=["context", "question"]
#     )

In [87]:
# PROMPT

In [89]:
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    chain_type="stuff",
    input_key="query",
    # chain_type_kwargs={"prompt": PROMPT},
    
)

#  Ask the Query 
query = "give me the best tata trucks under 10lakh??"

# 2nd Solution.
print(qa_chain(query)['result'])
# print(type(qa_chain(query)))

 I'm sorry, I cannot provide an accurate answer as none of the trucks listed have a price under 10 lakh. 


# 91_Trucks_Chatbot_Demo_USING_CSV

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import warnings 
warnings.filterwarnings("ignore")

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# Create the OpenAI Chat Model instance
llm = ChatOpenAI(temperature=0.7)

llm.invoke("Who is Elon Musk in detail in humanizer format.").content

# Import the CSV File.

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="trucks_data.csv", source_column="prompt")

# data = loader.load()[0].metadata['source']
data = loader.load()

In [ ]:
# data[0].page_content
data

# Perform the Embedding of the CSV Data

In [ ]:
# # This is working Fine* ******************************************************************
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Use a pre-trained SentenceTransformer model
embeddings = OpenAIEmbeddings()
 
# Create FAISS vector store from documents
vectordb = FAISS.from_documents(documents=data, embedding=embeddings)

# Optionally: Save the FAISS index to a local file for later use
vectordb.save_local("faiss_index")

# Create a retriever from the vector store..
retriever = vectordb.as_retriever()


# query = "for how long is this course Valid?"  # Example query
query = "how about job placement support"  # Example query

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


In [ ]:
# RetrievalQA Class....
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff",retriever=retriever, input_key="query", return_source_documents = True, chain_type_kwargs = chain_type_kwargs)

# chain("Did you provide the internship tell me in detail?")
chain("Is 91trucks business model is scalable?")['result']